<a href="https://colab.research.google.com/github/rubyvanrooyen/notebooks/blob/main/getting_started/TART_cmd_calibrate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TART command line tools

Using the command line tools from the `tart-tools` package to interface with a TART telescope and obtain calibration data

Extracted from the TART continuous calibration script `tart_calibrate.sh` authored by Tim Molteno.

Reference
Molteno et al. "Continuous Calibration of the Transient Array Radio Telescope using Satellites" from ENZCon 2017

## Installation

PyPI links for installation instructions and package information:
* https://pypi.org/project/tart/
* https://pypi.org/project/tart-tools/

In [ ]:
!pip install tart tart-tools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 105 kB 4.3 MB/s 
     |████████████████████████████████| 23.7 MB 25.3 MB/s 


## TART calibration

### Get calibration data

In [ ]:
!tart_calibration_data -h

usage: tart_calibration_data [-h] [--api API] [--file FILE] [--n N]
                             [--interval INTERVAL] [--catalog CATALOG]

Generate Calibration Data.

optional arguments:
  -h, --help           show this help message and exit
  --api API            Telescope API server URL. (default:
                       https://tart.elec.ac.nz/signal)
  --file FILE          Calibration Output JSON file. (default:
                       calibration_data.json)
  --n N                Number of samples in the JSON data file. (default: 5)
  --interval INTERVAL  Interval (in minutes) between samples in the JSON data
                       file. (default: 5)
  --catalog CATALOG    Catalog API URL. (default:
                       https://tart.elec.ac.nz/catalog)


E.g. `tart_calibration_data --api <API> --n 3 --i 25 --file <CALIB_DATA_FILE>`

In [ ]:
API = "https://tart.elec.ac.nz/signal/"
CALIB_DATA = "calib_data.json"

In [ ]:
!tart_calibration_data --api {API} --file {CALIB_DATA} --n 1

Loading new data from https://tart.elec.ac.nz/signal/...
Getting catalog from https://tart.elec.ac.nz/catalog/catalog?lat=-45.85177&lon=170.5456&date=2022-10-13T08:35:10.639000+00:00
Loading Complete


### Perform optimization

In [ ]:
!tart_calibrate -h

usage: tart_calibrate [-h] [--api API] [--file FILE] [--show] [--cold-start]
                      [--get-gains] [--dir DIR] [--method METHOD]
                      [--iterations ITERATIONS] [--elevation ELEVATION]
                      [--ignore IGNORE [IGNORE ...]]

Calibrate the tart telescope from API.

optional arguments:
  -h, --help            show this help message and exit
  --api API             Telescope API server URL. (default:
                        https://tart.elec.ac.nz/signal)
  --file FILE           Calibration Output JSON file. (default:
                        calibration_data.json)
  --show                show instead of save. (default: False)
  --cold-start          Start from zero knowledge of gains. (default: False)
  --get-gains           Start from current knowledge of gains. (default:
                        False)
  --dir DIR             Output directory. (default: .)
  --method METHOD       Optimization Method [NM, LB, DE, BH] (default: LB)
  --iterations

E.g. `tart_calibrate --api <API> --file <CALIB_DATA> --method <METHOD>`

In [ ]:
METHOD = "BH"
# calibration output file produced by tart_calibrate
CALIB_OUTPUT = f"{METHOD}_opt_json.json"

In [ ]:
!tart_calibrate --api {API} --file {CALIB_DATA} --method {METHOD}

Streaming output truncated to the last 5000 lines.
Iteration 422100, score=-15.02
S/N -4.04, ZONE: -10.99
Iteration 422200, score=-15.03
S/N -4.04, ZONE: -10.99
Iteration 422300, score=-15.03
S/N -4.04, ZONE: -10.99
Iteration 422400, score=-15.03
S/N -4.04, ZONE: -10.99
Iteration 422500, score=-15.03
S/N -4.04, ZONE: -10.99
Iteration 422600, score=-15.03
basinhopping step 245: f -16.471 trial_f -15.0283 accepted 0  lowest_f -16.7217
BH f=-15.028260953968186 accepted 0
{
    "gain": [
        1.0,
        0.9104,
        1.2385,
        0.8487,
        1.3283,
        1.16,
        1.2081,
        1.308,
        0.8783,
        1.2924,
        1.3357,
        1.3069,
        1.6086,
        1.2038,
        0.8175,
        0.7236,
        1.2753,
        1.1261,
        1.207,
        1.1305,
        0.5454,
        0.9956,
        1.2827,
        1.2669
    ],
    "rot_degrees": 1.7102807715520518,
    "phase_offset": [
        0.0,
        0.3854,
        -2.0692,
        -0.1581,
    

In [ ]:
import os
import shutil  
print(os.getcwd())

/content


In [ ]:
from datetime import datetime
t = datetime.now()
DATESTR = t.strftime('%Y_%m_%d_%H_%M_%S')

In [ ]:
source = CALIB_OUTPUT
destination = f"cal_{DATESTR}.json"
shutil.move(source, destination)
print(f"Calibration output is in {destination}")

Calibration output is in cal_2022_10_13_09_05_32.json


### Upload gains to telescope

E.g. `tart_upload_gains --api <API> --gains <destination> --pw <password>`

NOTE. Modify the <password> to be the password for your telescope to allow the gains and phases to be saved.

In [ ]:
!tart_upload_gains -h

usage: tart_upload_gains [-h] [--api API] [--pw PW] --gains GAINS

Upload gains from local file to api

optional arguments:
  -h, --help     show this help message and exit
  --api API      Telescope API server URL. (default:
                 https://tart.elec.ac.nz/signal)
  --pw PW        API password (default: password)
  --gains GAINS  local file to upload (default: None)
